In [1]:
import statsapi as mlb
import pandas as pd
import sqlalchemy
import config
import csv
from os import walk
import re
from datetime import datetime
import functions
import sql_alch_schema
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String,DateTime,Date,Time,Boolean,Float
from sqlalchemy import ForeignKey,and_
from sqlalchemy.orm import relationship


_sql_alchemy_connection = (
                                f'mysql+mysqlconnector://'
                                f'{config.user}:{config.password}'
                                f'@{config.host}:{config.port}'
                                f'/{config.schema}'
                           )
## Create the engine 
db = sqlalchemy.create_engine(_sql_alchemy_connection,
                              echo = False,
                              connect_args = {'ssl_disabled' : True,})

Base = sql_alch_schema.Base

Game = sql_alch_schema.Game
Play = sql_alch_schema.Play
Team = sql_alch_schema.Team
GameTeamLink = sql_alch_schema.GameTeamLink
Person = sql_alch_schema.Person
PersonPlayLink=sql_alch_schema.PersonPlayLink



from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db)
session = Session()


## pitches

It's still a stretch goal to incorporate pitch data, runners data, and action data. this is the beginning of that work

In [2]:
def get_pitches(api_call):
    """
    Takes in the result of of mlbstats-api call on the game endpoint. 
    Returns list of dictionaries for each play
    """
    gameData = api_call['gameData']
    game = gameData['game']
    
    liveData = api_call['liveData']
    
# 'play_test' is meant to represent one of the dictionaries that represent each play in the 'game' endpoint  

[play_test['playEvents'][x] for x in play_test['pitchIndex']]

## Actions 

[play_test['playEvents'][x] for x in play_test['actionIndex']]

## Runners

len(play_test['runners'])

play_test['runners'][0]['movement']['start']

[play_test['runners'][x]['movement']['start'] for x in play_test['runnerIndex']]

NameError: name 'play_test' is not defined

## Investigate a more effecient way of getting player's performance up to a game

In [3]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db)
session = Session()

In [4]:
# this is what we have now
query = session.query(Game,Play).\
                filter(Game.id==Play.game_id).\
                filter(and_(Game.type=='R',Game.dateTime<date)).\
                filter(Play.pitcher_id==self.id).\
                order_by(Game.dateTime.desc()).\
                all()[:limit]

NameError: name 'date' is not defined

In [ ]:
def player_totals(self,game_record,session,limit=1000):    
    import numpy as np
    date=datetime.strftime(game_record.dateTime,'%Y%m%d')
    
    probable_starters = [game_record.home_probablePitcher,game_record.away_probablePitcher]
    stat_line = {}
    
    if self.primaryPosition_type in ['Hitter','Outfielder','Infielder','Catcher']:
        query = session.execute(f"""select
            g.id,
            p.id,
            p.`event`,
            p.halfInning,
            p.inning,
            p.atBatIndex,
            p.hasOut,
            p.RBI,
            p.count_outs,
            p.num_pitches
        from 
            MLB_Stats.games g
        right join
            MLB_Stats.plays p
            on 
            g.id=p.game_id
        where
            p.batter_id={self.id}
            and 
            date(g.`dateTime`)<date({date})
            and
            g.type='R'
        order by 
            p.startTime desc;""").fetchall()
    
    if self.primaryPosition_type=='Pitcher':
        query = session.execute(f"""select
            g.id,
            p.id,
            p.`event`,
            p.halfInning,
            p.inning,
            p.atBatIndex,
            p.hasOut,
            p.RBI,
            p.count_outs,
            p.num_pitches
        from 
            MLB_Stats.games g
        right join
            MLB_Stats.plays p
            on 
            g.id=p.game_id
        where
            p.pitcher_id={self.id}
            and 
            date(g.`dateTime`)<date({date})
            and
            g.type='R'
        order by 
            p.startTime desc;""").fetchall()
        
    plays=([
        {'game_id':q[0],'play_id':q[1],
         'event':q[2],'halfInning':q[3],
         'inning':q[4],'atBatIndex':q[5],
         'hasOut':q[6],'RBI':q[7],
         'count_outs':q[8],
         'num_pitches':q[9]} 
        for q in query])
    games={x[0] for x in query}

    
    singles = [x for x in plays if x['event']=='Single']
    doubles = [x for x in plays if x['event']=='Double']
    triples = [x for x in plays if x['event']=='Triple']
    home_runs = [x for x in plays if x['event']=='Home Run']
    walks = [x for x in plays if x['event']=='Walk']
    strikeouts = [x for x in plays if x['event'] in ['Strikeout','Strikeout Double Play']]
    HBP = [x for x in plays if x['event']=='Hit By Pitch']
    GDP = [x for x in plays if x['event']=='Grounded Into DP']
    IBB = [x for x in plays if x['event']=='Intent Walk']
    sac = [x for x in plays if x['event'] in ['Sac Bunt','Sac Fly']]
    interference = [x for x in plays if x['event']=='Catcher Interference']
    LO = [x for x in plays if x['event']=='Lineout']
    PO = [x for x in plays if x['event']=='Pop Out']
    FO = [x for x in plays if x['event']=='Flyout']
    GO = [x for x in plays if x['event']=='Groundout']
    
    ## pitcher stats 
    #starts
    #for hitters, this will mean they're the leadoff batter
    GS = {(x['inning'],x['game_id']) for x in plays if x['inning']==1}
    #games finished isn't adding up correctly yet. I think it's because walk-offs aren't counted
#     GF = {(x['inning'],x['game_id']) for x in plays 
#           if x['inning']==max([p.inning for p in x.game.plays]) 
#           and x.count_outs==3}        
    plays_with_outs = [x for x in plays if x['hasOut']==True]
    double_plays = [x for x in plays if x['event']=='Grounded Into DP' or 'Double Play' in x['event']]
    triple_plays = [x for x in plays if x['event']=='Triple Play']
    outs = len(plays_with_outs)+len(double_plays)+(len(triple_plays)*2)
    
    stat_line['Position_type']=self.primaryPosition_type
    stat_line['games']=len(games)
    stat_line['PA'] = len(plays)
    stat_line['AB'] = len(plays)-len(walks)-len(HBP)-len(IBB)-len(sac)-len(interference)
    stat_line['hits']=len(singles)+len(doubles)+len(triples)+len(home_runs)
    stat_line['singles']=len(singles)
    stat_line['doubles']=len(doubles)
    stat_line['triples']=len(triples)
    stat_line['home_runs']=len(home_runs)
    stat_line['walks']=len(walks)
    stat_line['strikeouts']=len(strikeouts)
    stat_line['HBP']=len(HBP)
    stat_line['RBIs']=sum([x['RBI'] for x in plays])
    stat_line['GDP']=len(GDP)
    stat_line['IBB']=len(IBB)
    stat_line['sac']=len(sac)
    stat_line['LO']=len(LO)
    stat_line['PO']=len(PO)
    stat_line['FO']=len(FO)
    stat_line['GO']=len(GO)
    stat_line['pitches_perPA']=np.mean([x['num_pitches'] for x in plays])
    
    stat_line['GS']=len(GS)
    #stat_line['GF']=len(GF)
        
    if 'Pitcher' in stat_line['Position_type']:
        stat_line['IP']=outs/3
    # is this pitcher generally a starter?    
    # pitchers who start the game less than 12th of their appearances are labeled relief
        if stat_line['GS']<stat_line['games']/12:
            stat_line['Position_type']='Pitcher_relief'
        else:
            stat_line['Position_type']='Pitcher_starter'
    
    # is this pitcher TODAY's starter?
    if self.id in probable_starters:
        stat_line['Position_type']='Probable_startingPitcher'            
    
    if 'Pitcher' not in stat_line['Position_type']:
        stat_line['IP']=0
    #saves/holds
    #stats for relief pitchers
    #stat_line['save_situations']=len([x for x in inning_score])       
    return stat_line

In [ ]:
Base.metadata.clear()

In [ ]:
class Cumulative_Stats(Base):
    __tablename__ = 'cumulative_stats'
    __table_args__ = {'extend_existing': True}
    
    player_id = Column(Integer,ForeignKey('people.id'),primary_key=True)
    game_id = Column(String(150),ForeignKey('plays.game_id'),primary_key=True)
    
    position_type = Column(String(20))
    games = Column(Integer)
    PA = Column(Integer)
    AB = Column(Integer)
    hits = Column(Integer)
    singles = Column(Integer)
    doubles = Column(Integer)
    triples = Column(Integer)
    home_runs = Column(Integer)
    walks = Column(Integer)
    strikeouts = Column(Integer)
    HBP = Column(Integer)
    RBIs = Column(Integer)
    GDP = Column(Integer)
    IBB = Column(Integer)
    sac = Column(Integer)
    LO = Column(Integer)
    PO = Column(Integer)
    FO = Column(Integer)
    GO = Column(Integer)
    pitches_perAB = Column(Integer)
    GS = Column(Integer)
    IP = Column(Integer)
    
    play = relationship('Play',back_populates='people')
    player = relationship('Person',back_populates='plays',lazy='dynamic')
    
    def __repr__(self): 
        return "<Game(pk='%s',id='%s')>" % (
                        self.player_id, self.game_id)
    
        
Person.plays = relationship("Cumulative_Stats",order_by=Play,back_populates='player',lazy='dynamic')
Play.people = relationship("Cumulative_Stats",back_populates='play')

In [ ]:
Base.metadata.clear()

In [ ]:
batters= session.execute("""select
                                plays.batter_id,
                                people.primaryPosition_type,
                                plays.game_id,
                                plays.startTime
                            from 
                                MLB_Stats.plays
                            inner join
                                MLB_Stats.people
                                on 
                                plays.batter_id=people.id
                            order by 
                                plays.startTime desc; """).fetchall()

In [ ]:
def create_addCumulative_Stats(session,row,limit=1000):    
    import numpy as np
    from datetime import datetime
    player_id = row[0]
    position = row[1]
    game_id = row[2]
    date = datetime.strftime(row[-1],"%Y%m%d")

    
    #probable_starters = [game_record.home_probablePitcher,game_record.away_probablePitcher]
    stat_line = {'game_id':game_id,'player_id':player_id}
    
    if position in ['Hitter','Outfielder','Infielder','Catcher']:
        query = session.execute(f"""select
            g.id,
            p.id,
            p.`event`,
            p.halfInning,
            p.inning,
            p.atBatIndex,
            p.hasOut,
            p.RBI,
            p.count_outs,
            p.num_pitches
        from 
            MLB_Stats.games g
        right join
            MLB_Stats.plays p
            on 
            g.id=p.game_id
        where
            p.batter_id={player_id}
            and 
            date(g.`dateTime`)<date({date})
            and
            g.type='R'
        order by 
            p.startTime desc;""").fetchall()
    
    if position=='Pitcher':
        query = session.execute(f"""select
            g.id,
            p.id,
            p.`event`,
            p.halfInning,
            p.inning,
            p.atBatIndex,
            p.hasOut,
            p.RBI,
            p.count_outs,
            p.num_pitches
        from 
            MLB_Stats.games g
        right join
            MLB_Stats.plays p
            on 
            g.id=p.game_id
        where
            p.pitcher_id={player_id}
            and 
            date(g.`dateTime`)<date({date})
            and
            g.type='R'
        order by 
            p.startTime desc;""").fetchall()
        
    plays=([
        {'game_id':q[0],'play_id':q[1],
         'event':q[2],'halfInning':q[3],
         'inning':q[4],'atBatIndex':q[5],
         'hasOut':q[6],'RBI':q[7],
         'count_outs':q[8],
         'num_pitches':q[9]} 
        for q in query])
    games={x[0] for x in query}

    
    singles = [x for x in plays if x['event']=='Single']
    doubles = [x for x in plays if x['event']=='Double']
    triples = [x for x in plays if x['event']=='Triple']
    home_runs = [x for x in plays if x['event']=='Home Run']
    walks = [x for x in plays if x['event']=='Walk']
    strikeouts = [x for x in plays if x['event'] in ['Strikeout','Strikeout Double Play']]
    HBP = [x for x in plays if x['event']=='Hit By Pitch']
    GDP = [x for x in plays if x['event']=='Grounded Into DP']
    IBB = [x for x in plays if x['event']=='Intent Walk']
    sac = [x for x in plays if x['event'] in ['Sac Bunt','Sac Fly']]
    interference = [x for x in plays if x['event']=='Catcher Interference']
    LO = [x for x in plays if x['event']=='Lineout']
    PO = [x for x in plays if x['event']=='Pop Out']
    FO = [x for x in plays if x['event']=='Flyout']
    GO = [x for x in plays if x['event']=='Groundout']
    
    ## pitcher stats 
    #starts
    #for hitters, this will mean they're the leadoff batter
    GS = {(x['inning'],x['game_id']) for x in plays if x['inning']==1}
    #games finished isn't adding up correctly yet. I think it's because walk-offs aren't counted
#     GF = {(x['inning'],x['game_id']) for x in plays 
#           if x['inning']==max([p.inning for p in x.game.plays]) 
#           and x.count_outs==3}        
    plays_with_outs = [x for x in plays if x['hasOut']==True]
    double_plays = [x for x in plays if x['event']=='Grounded Into DP' or 'Double Play' in x['event']]
    triple_plays = [x for x in plays if x['event']=='Triple Play']
    outs = len(plays_with_outs)+len(double_plays)+(len(triple_plays)*2)
    
    stat_line['Position_type']=position
    stat_line['games']=len(games)
    stat_line['PA'] = len(plays)
    stat_line['AB'] = len(plays)-len(walks)-len(HBP)-len(IBB)-len(sac)-len(interference)
    stat_line['hits']=len(singles)+len(doubles)+len(triples)+len(home_runs)
    stat_line['singles']=len(singles)
    stat_line['doubles']=len(doubles)
    stat_line['triples']=len(triples)
    stat_line['home_runs']=len(home_runs)
    stat_line['walks']=len(walks)
    stat_line['strikeouts']=len(strikeouts)
    stat_line['HBP']=len(HBP)
    stat_line['RBIs']=sum([x['RBI'] for x in plays])
    stat_line['GDP']=len(GDP)
    stat_line['IBB']=len(IBB)
    stat_line['sac']=len(sac)
    stat_line['LO']=len(LO)
    stat_line['PO']=len(PO)
    stat_line['FO']=len(FO)
    stat_line['GO']=len(GO)
    stat_line['pitches_perPA']=np.mean([x['num_pitches'] for x in plays])
    
    stat_line['GS']=len(GS)
    #stat_line['GF']=len(GF)
        
    #if 'Pitcher' in stat_line['Position_type']:
        #stat_line['IP']=outs/3
    # is this pitcher generally a starter?    
    # pitchers who start the game less than 12th of their appearances are labeled relief
    if stat_line['GS']<stat_line['games']/12:
        stat_line['Position_type']='Pitcher_relief'
    else:
        stat_line['Position_type']='Pitcher_starter'
    
#     is this pitcher TODAY's starter?
#     if player_id.id in probable_starters:
#         stat_line['Position_type']='Probable_startingPitcher'            
    
    if 'Pitcher' not in stat_line['Position_type']:
        stat_line['IP']=0
    
    return stat_line

In [ ]:
session.rollback()

In [ ]:
stat_lines=[]
for chunk in sql_alch_schema.chunk(50,batters):
    count=1
    print(f'chunk{count} out of {batters}')
    stat_lines.append([create_addCumulative_Stats(session,x) for x in chunk])
    count+=1

In [ ]:
stat_line

In [ ]:
Cumulative_Stats(    
    player_id = stat_line['player_id']
    game_id = stat_line['game_id']
    
    position_type = stat_line['Position_type']
    games = stat_line['games']
    PA = stat_line['PA']
    AB = stat_line['AB']
    hits = stat_line['hits']
    singles = stat_line['singles']
    doubles = stat_line['doubles']
    triples = stat_line['triples']
    home_runs = stat_line['']
    walks = Column(Integer)
    strikeouts = Column(Integer)
    HBP = Column(Integer)
    RBIs = Column(Integer)
    GDP = Column(Integer)
    IBB = Column(Integer)
    sac = Column(Integer)
    LO = Column(Integer)
    PO = Column(Integer)
    FO = Column(Integer)
    GO = Column(Integer)
    pitches_perAB = Column(Integer)
    GS = Column(Integer)
    IP = Column(Integer))

In [5]:
bartolo_colon = session.query(Person).first()

In [14]:
bartolo_colon.plays.first()

In [13]:
bartolo_colon_play = session.query(Play).filter(Play.pitcher_id==bartolo_colon.id).first()

In [ ]:
person_play = (bartolo_colon.id,bartolo_colon_play.id)

In [2]:
people_plays=db.execute("""select batter_id,id from plays;""").fetchall()

people_plays=[tuple(x.values()) for x in people_plays]

In [3]:
existing=db.execute("""select * from people_plays""").fetchall()

existing=[tuple(x.values()) for x in existing]

In [4]:
from sqlalchemy.sql.expression import insert

def chunk_insertPeoplePlayLink(db,chunk_size=5000):
    people_plays=db.execute("""select batter_id,id from plays;""").fetchall()
    people_plays=[tuple(x.values()) for x in people_plays]
    
    existing=db.execute("""select * from people_plays""").fetchall()
    existing=[tuple(x.values()) for x in existing]
    
    people_plays=[x for x in people_plays if x not in existing]
    
    count=1
    chunks = sql_alch_schema.chunk(chunk_size,people_plays)
    for chunk_ in chunks:
        print(f"chunk {count} of {len(chunks)}")
        insert_stmt=insert(PersonPlayLink,chunk_)
        db.execute(insert_stmt)
        count+=1


In [ ]:
chunk_insertPeoplePlayLink(db)

chunk 1 of 83
chunk 2 of 83
chunk 3 of 83
chunk 4 of 83
chunk 5 of 83
chunk 6 of 83
chunk 7 of 83
chunk 8 of 83
